In [3]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import math

In [4]:
os.chdir('C:\\Users\\DELL\\Documents\\Semester_2\\DMA\\Homework 3')

In [5]:
df = pd.read_excel("sub-variant.xlsx")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'sub-variant.xlsx'

In [ ]:
df["Actual Class"].unique()

In [ ]:
df['Actual Class'] = df['Actual Class'].map({'O': 0, 'NV': 1})
df.rename(columns = {'Probability of New Variant (NV) from the testing laboratory':'propensity'}, inplace = True)
df = df.sort_values(by = "propensity",axis=0, ascending=False)
df.head()

In [ ]:
def Classification_Metrics(df,p):
    df1 = df[df["propensity"]>p]
    df2 = df[df["propensity"]<p]
    n11 = df1[df1["Actual Class"]==1]["Actual Class"].count()
    #print("n11 ", n11)
    n21 = df1[df1["Actual Class"]==0]["Actual Class"].count()
    #print("n21 ", n21)
    n22 = df2[df2["Actual Class"]==0]["Actual Class"].count()
    #print("n22 ", n22)
    n12 = df2[df2["Actual Class"]==1]["Actual Class"].count()
    #print("n12 ", n12)
    F1 = 2*n11/(2*n11+n21+n12)
    MCC = MCC_Cal(n11,n12,n21,n22)
    #MCC = ((n11*n22)-(n21*n12))/math.sqrt((n11+n21)*(n11+n12)*(n21+n22)*(n12+n22))
    #print("F1 ", F1)
    #print("MCC ", MCC)
    sensitivity = n11/(n11+n12)
    specificity = 1 - (n22/(n21+n22))
    return sensitivity,specificity,F1,MCC

In [ ]:
def MCC_Cal(TP,FN,FP,TN):
    if TP == 0 and FP == 0:
        if FN == 0 and TN != 0:
            MCC = 1
        elif FN != 0 and TN != 0:
            MCC = 0
        else:
            MCC = -1
    elif TP == 0 and FN == 0:
        if FP == 0 and TN != 0:
            MCC = 1
        elif FP != 0 and TN != 0:
            MCC = 0
        else:
            MCC = -1
    elif TN == 0 and FP == 0:
        if TP != 0 and FN == 0:
            MCC = 1
        elif TP != 0 and FN != 0:
            MCC = 0
        else:
            MCC = -1
    elif TN == 0 and FN == 0:
        if TP != 0 and FP == 0:
            MCC = 1
        elif TP != 0 and FP != 0:
            MCC = 0
        else:
            MCC = -1
    else:
        MCC = ((TP*TN)-(FP*FN))/math.sqrt((TP+FP)*(TP+FN)*(FP+TN)*(FN+TN))
    return MCC
        

In [ ]:
pro=[0,0.2,0.4,0.5,0.6,0.8,1]
sensitivity = []
specificity = []
F1_Score = []
MCC = []
for p in pro:
    a,b,c,d = Classification_Metrics(df,p)
    sensitivity.append(a)
    specificity.append(b) 
    F1_Score.append(c)
    MCC.append(d)
table = pd.DataFrame({"Cutoff":pro, "Sensitivity":sensitivity, "1 - Specificity":specificity, "F1_Score":F1_Score, "MCC":MCC})
table

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(x=[0, 1], y=[0,1]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=specificity, y=sensitivity),
    row=1, col=1
)

fig.update_layout(height=600, width=800, title_text='ROC Curve',xaxis_title="1 - Specificity",yaxis_title="Sensitivity")
fig.show()


#fig = px.line(df, x=specificity, y=sensitivity, title='ROC Curve')
#fig.show()

a) From the graph we take Sensitivity = 0.87 and Specificity = 0.74 for an optimal cutoff is 0.4
b) MCC = 0.550877 and F1_Score = 0.764706 for a recommednded balanced cutoff = 0.4 
MCC = 0.522807 and F1_Score = 0.733333 for a defaut cutoff = 0.5

MCC and F1_Score improved for the recommended cutoff than that of default cutoff which is a positive light on model perfomance and usability

c) In this scenario, the goal is to assess the classifier's ability to distinguish between Omicron and the new variant. Since we are interested in identifying new variant cases with high sensitivity, we require a measure that prioritizes sensitivity or recall, such as F-score. As a result, using F-score to evaluate the classifier's performance is recommended.

### 2

In [ ]:
n11 = 38950
n12 = 1050
n21 = 1500
n22 = 58500
Accuracy = (n11+n22)/(n11+n12+n21+n22)
Error_rate = 1-Accuracy
sensitivity = n11/(n11+n12)
specificity = 1 - (n22/(n21+n22))
print("Error_rate ",Error_rate)
print("Sensitivity ",sensitivity)
print("Specificity ",specificity)

In [ ]:
n11 = 2500
n12 = 1250
n21 = 1275
n22 = 4975
Accuracy = (n11+n22)/(n11+n12+n21+n22)
Error_rate = 1-Accuracy
sensitivity = n11/(n11+n12)
specificity = 1 - (n22/(n21+n22))
print("Error_rate ",Error_rate)
print("Sensitivity ",sensitivity)
print("Specificity ",specificity)
#overfitting

There appears to have a overfitting model on validation datasetwhich can be seen from the data,

On the training set, the model achieved a very low error rate of 0.0255, indicating that it was able to accurately classify a large majority of the cases in the dataset. The sensitivity, which measures the proportion of true positive cases that were correctly identified, was very high at 0.97375, indicating that the model was able to correctly identify almost all of the Diabetic retinopathy cases in the training set. However, the specificity, which measures the proportion of true negative cases that were correctly identified, was very low at 0.025, indicating that the model incorrectly identified a large proportion of normal cases as Diabetic retinopathy cases.

On the validation set, however, the model performed much worse. The error rate was much higher at 0.2525, indicating that the model incorrectly classified a large proportion of cases in the validation set. The sensitivity was still moderate at 0.6667, indicating that the model was able to correctly identify a reasonable proportion of the Diabetic retinopathy cases in the validation set. However, the specificity was still quite low at 0.204, indicating that the model incorrectly identified a large proportion of normal cases as Diabetic retinopathy cases in the validation set.


### 4

In [ ]:
bc = pd.read_excel("Bostonconsultancy.xlsx")
bc.head()

In [ ]:
bc.rename(columns = {'Probability of winning the opportunity':'propensity'}, inplace = True)
bc = bc.sort_values(by = "propensity",axis=0, ascending=False)
bc.head()

In [ ]:
bc["Gain, C(k)"] = bc["Actual Class"].cumsum()
bc.head()

In [ ]:
bc.shape[0]

In [ ]:
lst = list(range(1,bc["Actual Class"].count()+1))
bc["Case, K"] = lst
bc.head()

In [ ]:
y = bc["Actual Class"].sum()/bc["Actual Class"].count()
bc["Baseline, A(k)"] = y*bc["Case, K"]
bc.head()

In [ ]:
bc["Lift, L(k)"] = bc["Gain, C(k)"]/bc["Baseline, A(k)"]
bc["Baseline, B(k)"] = bc["Baseline, A(k)"]/bc["Baseline, A(k)"]
bc.head()

In [ ]:
bc["Service Cost"] = bc["Service Cost"].apply(lambda x: x.replace("$", ""))
bc["Service Cost"] = bc["Service Cost"].astype('int64')
bc.head()

In [ ]:
Cost = 60+75+100+50+50
#bc["Net Profit"] = (bc["Service Cost"] - Cost).cumsum()
bc["Net Profit"] = np.where(bc["Actual Class"]==1, (bc["Service Cost"] - Cost), -Cost).cumsum()
bc.head()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(mode="lines+markers",x=bc["Case, K"], y=bc["Baseline, A(k)"]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(mode="lines+markers",x=bc["Case, K"], y=bc["Gain, C(k)"]),
    row=1, col=1
)


fig.update_layout(height=600, width=800, title_text='Gain Chart',xaxis_title="Case #",yaxis_title="Cum Gain")
fig.show()


#fig = px.line(df, x=specificity, y=sensitivity, title='ROC Curve')
#fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(mode="lines+markers",x=bc["Case, K"], y=bc["Net Profit"]),
    row=1, col=1
)

fig.update_layout(height=600, width=800, title_text='Net Profit',xaxis_title="Case #",yaxis_title="Cum Net Profit")
fig.show()


#fig = px.line(df, x=specificity, y=sensitivity, title='ROC Curve')
#fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(mode="lines+markers",x=bc["Case, K"], y=bc["Baseline, B(k)"]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(mode="lines+markers",x=bc["Case, K"], y=bc["Lift, L(k)"]),
    row=1, col=1
)

fig.update_layout(height=600, width=800, title_text='Lift Chart',xaxis_title="Case #",yaxis_title="Lift")
fig.show()


#fig = px.line(df, x=specificity, y=sensitivity, title='ROC Curve')
#fig.show()

In [ ]:
bc1 = pd.read_excel("Bostonconsultancy.xlsx")
bc1.head()

In [ ]:
bc1.rename(columns = {'Probability of winning the opportunity':'propensity'}, inplace = True)
bc1 = bc1.sort_values(by = "propensity",axis=0, ascending=False)
bc1.head()

In [ ]:
bc1.shape

In [ ]:
bc1.drop(bc1.tail(bc1.shape[0]%10).index,inplace=True)
bc1.shape

In [ ]:
bc1.head()

As we are allowed to do 10 discovery calls per month 
We have divided the dataset with 10 Companies per group instead of having 7 records per group(70/10) which is usually the case in generic Decile Chart.

In [ ]:
bc1['Decile'] = pd.qcut(bc1['propensity'], 7, labels=[i for i in range(7,0,-1)])
bc1

In [ ]:
bc1["Service Cost"] = bc1["Service Cost"].apply(lambda x: x.replace("$", ""))
bc1.head()

In [ ]:
bc1["Service Cost"] = bc1["Service Cost"].astype('int64')

In [ ]:
bc2=pd.DataFrame(bc1.groupby(["Decile"])["Actual Class"].mean())
bc2 = bc2.sort_index(ascending=False)
bc2.rename(columns={"Actual Class":"Dd"},inplace=True)
bc2.head()

In [ ]:
bc2["y1"]=bc1["Actual Class"].mean()
bc2

In [ ]:
bc2["Ld"] = bc2["Dd"]/bc2["y1"]
bc2

In [ ]:
fig = px.bar(bc2, x=bc2.index, y='Ld', color='Ld')
fig.add_hline(y=1)
fig.update_layout(height=600, width=800, title_text='Decile Chart')
fig.show()

In [ ]:
bc1[bc1["Decile"]==1]

From this we can see that the clients they are to reach out to following clients
- Amazon.com
- Charter Communications
- AIG
- MetLife
- Citigroup
- PepsiCo
- Bank of America
- Morgan Stanley	
- Ford Motor	
- Home Depot
because by following the recommendation we get lift of 1.44

c) The top recommended company by the model belongs to USA and using GCP Big Query and Looker BI Tool majorly and with an average cloud data storage size of more than 500TB